In [65]:
import requests
import telepot
import datetime
import time
import pandas as pd
import numpy as np
from skpy import Skype

import string
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [52]:
LOGIN_URL = 'https://admin-api.dd8tvp2.com/login'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
            'accept-language': 'zh-TW'}
payload = {'password': '#qaz&123',
            'username': 'torin',}
#發送請求登入
session_requests = requests.session()
response = session_requests.request('POST',url=LOGIN_URL, data=payload, headers=headers)
token_aa=response.json()['access_token']
print(response)

<Response [200]>


In [53]:
#取得 token
headerss = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
            'accept-language': 'zh-TW','authorization':'Bearer '+token_aa}
#抓取总页数
ttt = session_requests.get("https://admin-api.dd8tvp2.com/video?site_code=doudou&page=1&count=1" , headers=headerss )
total_count=ttt.json()['paging']['total']
total_count=int(total_count)

In [54]:
list_data_all=[]
for j in range(1,int(total_count/100)+2,1): 
    ttt1 = session_requests.get("https://admin-api.dd8tvp2.com/video?site_code=doudou&page="+str(j)+"&count=100" , headers=headerss )
    output_data=ttt1.json()['data']
    list_data_all=list_data_all+output_data

In [61]:
df_dodo_data=pd.DataFrame(list_data_all)[['lib_video_id','title','title_en','category_name','play','favorite','type','price','buy','update_at']]
df_dodo_data['type_price']=df_dodo_data['type'].map({1: 'VIP观影限定',0:'免费'})
df_dodo_data=df_dodo_data[['lib_video_id','title','title_en','category_name','play','favorite','type_price','price','buy','update_at']]
df_dodo_data.columns=['片庫編號','A片名','番號','主題','播放數','收藏數','收費方式','價格','購買數','更新時間']
df_dodo_data=df_dodo_data.sort_values(by=['播放數'], ascending=False)

In [69]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly','https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/spreadsheets']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
service = build('sheets', 'v4', credentials=creds)
# Call the Sheets API
sheet = service.spreadsheets()
#寫入 data 進入 google sheet #美東時間
def insert_data(value_range_body,sheetname,colname1,colname2,col1,col2) :
    SAMPLE_RANGE_NAME = str(sheetname)+'!'+colname1+str(col1)+':'+colname2+str(col2)
    request= sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,valueInputOption='USER_ENTERED',range=SAMPLE_RANGE_NAME , body=value_range_body).execute()
def clear_data(sheetname,colname1,colname2,col1,col2):
    SAMPLE_RANGE_NAME = str(sheetname)+'!'+colname1+str(col1)+':'+colname2+str(col2)
    request = sheet.values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute() #, body=value_range_body

#Boin data insert
SAMPLE_SPREADSHEET_ID = '1VZku3NyOO37b2MgGPCjNIrscYki2XmyR9QfN9yBLalo'
clear_data('豆豆后台','A','J',2,10000)
value_range_body = {"majorDimension":"ROWS","values": df_dodo_data.to_dict('split')['data'] } 
insert_data(value_range_body,'豆豆后台','A','J',2,10000)

In [50]:
##數據輸出
#downloadpath = '//navdi/DEM_UserData/btorin/Desktop/fanza_data/'
#with pd.ExcelWriter(downloadpath+"dodo_video.xlsx" ) as dodo_video: #, index=False
#    df_dodo_data.to_excel(dodo_video, sheet_name="data", index=False)
#    dodo_video.save()

Calling close() on already closed file.


In [15]:
#sk.chats.recent() 
#sk.chats.chat('19:4f608eb0d3564058911ccc1961edd76a@thread.skype').sendFile(open("song.mp3", "rb"), "song.mp3") 